# A3: A\*, IDS, and Effective Branching Factor

### Josh Mau / jjmau14


In this assignment, I have implemented the Recursive Best-First Search implementation of the A\* algorithm. The base of the A\* algorithm was derived from Ph.D. of Computer Science, Chuck Anderson and is available at 
* http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/07%20Informed%20Search.ipynb 

This assignment compares three different heuristic functions used by the A\* algorithm to the `iterativeDeepeningSearch` algorithm implemented in the previous assignment. These heuristics and algorithms are compared by reporting the total number of nodes expanded, the depth of expansion, and the EBF (Effective Branching Factor) value of each. The EBF is essentially the average or expected number of successor nodes generated by the typical node for the given algorithm. 

### Implementation of EBF
Binary search was used to determine the EBF for any given algorithm. An initial starting position is determined given the range of possible values based on the depth and how many nodes were expanded. The following equation is then used to create a guess as to what the actual Effective Branching Factor may be (the value of **b**) and determining its proximity to **d** (depth of search). If the value if **b** is within the valid proximity of **d** (based on the precision passed to the function -- defaults to 0.01), then **b** is accepted as the EFB of the algorithm.

$$ \frac{1-b^{d+1}}{1-b}$$

The purpose of this function is to determine the effectiveness of the heuristic function used in the the A\* search algoritm.

### Functions implemented
The main functions used in this notebook are the following:

* `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
* `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
* `ebf(nNodes, depth, precision=0.01)`
   
Additional functions include those which define heuristic functions, or handle rules for the 8-puzzle game such as validity of moves, testing for goal state, and committing moves to the board. These functions are as follows:

* `actionsF_8p(state)`: returns a list of all possible valid actions given the current location (state) 
* `takeActionsF_8p(state, action)`: handles executing actions and updating the current state of the puzzle
* `goalTestF_8p(state, goal)`: returns a boolean value: is the puzzle complete or not
   
The main function of this assignment is `runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])`. This function takes a list of 3 valid goalStates given a previously specified startState [1, 2, 3, 4, 0, 5, 6, 7, 8]. The function first runs `iterativeDeepeningSearch` for each of the three goal states and produces the total nodes expanded, the depth of expansion, and the EBF. 


## Heuristic Functions

For `aStarSearch`, the following heuristics were used. 

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: Definition and proof below

## Explanation of Heuristic 3
The 3rd heuristic is uses the 0 based index to return a heuristic. Since the table is set configured with indexes:

$$
\begin{array}{ccc}
0 & 1 & 2\\
3 & 4 & 5\\
6 & 7 & 8
\end{array}
$$

the heuristic function will get the index of 0 in the state, and the index of 0 in the goal state. The two indexes are then subtracted from each other in no particular order, and the absolute value of this calulation (using math.fabs) is then divided by 2 using *specifically* integer division. This avoids overestimation. An example follows the first is the example current state, and the second is the example goal state:

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3\\
4 & 0 & 5  & & 4 & 7 & 5\\
6 & 7 & 8 &  & 6 & 0 & 8
\end{array}
$$

The index of 0 in the current state is index 4. The index of 0 in the goal state is 7. In this case, 7 is subtracted from 4, producing -3. ABS(-3) is 3. This result is then divided by 2 using integer division producing a result of 1.

1 is not an overestimation since the actual shortest path is 1. Another example follows:

$$
\begin{array}{ccccccccccc}
0 & 2 & 3  & ~~~~ & 2 & 3 & 5\\
4 & 1 & 5  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 7 & 0
\end{array}
$$

In this case, the index of 0 in the current state is 0, and the index of 0 in the goal state is 8. These two numbers would be subtracted and the absolute value will be taken resulting in 8 and a total distance. 8 is clearly an overestimation hence division by 2. The case in which the 0's are in the corners is the maximum case, there cannot be a greater distance between the two. 4 would be the result of the heuristic in this case which is the actual shortest path. 

## States used

The following 8-Puzzel was used to perform all tests in this assignment

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$


These known valid goal states are passed to the `runExperiment` function which tests each algorithm on each goal state and reports back stats

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

## Code 
Contains each function definition and test cases in the main 

In [34]:
import math

def iterativeDeepeningSearch(startState, goalState,
                             actionsF, takeActionF, maxDepth):
	global itsTotal
	global itsDepth
	itsTotal = 0
	itsDepth = 0
    
	for depth in range(maxDepth):
		result = depthLimitedSearch(startState, goalState,
                                    actionsF, takeActionF, depth)
		if result is 'failure':
			return 'failure'
		if result is not 'cutoff':
			result.insert(0, startState)
			itsDepth = depth
			return result
	return 'cutoff'


def depthLimitedSearch(state, goalState,
                       actionsF, takeActionF, depthLimit):
	global itsTotal
	if state == goalState:
		return []
	if depthLimit == 0:
		return 'cutoff'
	cutoffOccurred = False
	for action in actionsF(state):
		itsTotal += 1
		childState = takeActionF(state, action)[0]
		result = depthLimitedSearch(childState, goalState,
                                    actionsF, takeActionF, depthLimit-1)
		if result is 'cutoff':
			cutoffOccurred = True
		elif result is not 'failure':
			result.insert(0, childState)
			return result
	if cutoffOccurred:
		return 'cutoff'
	else:
		return 'failure'

######################################################################
# Eight Puzzle


def findBlank_8p(s):
	return iTorc(s.index(0))


def actionsF_8p(state):
	r, c = findBlank_8p(state)
	actions = []
	if c > 0:
		actions.append(('left', 1))
	if c < 2:
		actions.append(('right',1))
	if r > 0:
		actions.append(('up',1))
	if r < 2:
		actions.append(('down',1))
	return actions


def takeActionF_8p(state, action):
	import copy
	state = copy.deepcopy(state)
	r, c = findBlank_8p(state)
	dr = dc = 0
	if action[0] is 'left':
		dc -= 1
	elif action[0] is 'right':
		dc += 1
	elif action[0] is 'up':
		dr -= 1
	elif action[0] is 'down':
		dr += 1
	newr, newc = r+dr, c+dc
	setTile(state, r, c, getTile(state, newr, newc))
	setTile(state, newr, newc, 0)
	return (state, 1)


def goalTestF_8p(s, goalState):
	return s == goalState


def printPath_8p(start, goal, solutionPath):
	print('\n\nPath from')
	printState_8p(start)
	print('  to')
	printState_8p(goal)
	if solutionPath is 'cutoff':
		print('was not found due to depth cutoff.')
	elif solutionPath is 'failure':
		print('cannot be found.')
	else:
		print('  is', len(solutionPath), 'nodes long:')
		indent = 0
		for s in solutionPath:
			printState_8p(s, indent)
			print()
			indent += 1


def printState_8p(state, indent=0):
	st = list(map(str, state))
	st[st.index('0')] = '-'
	blanks = ' '*indent
	for i in range(0, 9, 3):
		print('{} {} {} {}'.format(blanks, st[0+i], st[1+i], st[2+i]))


def rcToi(row, col):
	return row*3+col


def iTorc(i):
	row = i // 3
	col = i - row*3
	return (row, col)


def setTile(state, row, col, tile):
	state[rcToi(row, col)] = tile
	return state


def getTile(state, row, col):
	return state[rcToi(row, col)]


class Node:
	def __init__(self, state, f=0, g=0 ,h=0):
		self.state = state
		self.f = f
		self.g = g
		self.h = h
	def __repr__(self):
		return "Node(" + repr(self.state) + ", f=" + repr(self.f) + ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def ebf(nNodes, depth, precision=0.01):
	if nNodes == depth:
		return 0
	if depth == 0:
		return 1.0
	n = depth/2.0
	minVal = 0.0
	maxVal = nNodes
	total = 0
    
	minRange = nNodes - (nNodes*precision)
	maxRange = nNodes + (nNodes*precision)
  
	while ( not(total > minRange and total < maxRange)):
		# 1 - (b^(d+1))/(1-b)
		total = ( (1.0-(n**(depth+1))) / (1.0-n) )

		if total < nNodes:
			minVal = n
			n = (n + maxVal) / 2.0
      
		if total > nNodes:
			maxVal = n
			n = (n + minVal) / 2.0
      
	return n
      
      
        
def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
	global aStarCounter
	global aStarDepth
	aStarDepth = 0
	aStarCounter = 0
	h = hF(startState)
	startNode = Node(state=startState, f=0+h, g=0, h=h)
	return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
	global aStarCounter
	global aStarDepth
	if goalTestF(parentNode.state):
		return ([parentNode.state], parentNode.g)
	## Construct list of children nodes with f, g, and h values
	actions = actionsF(parentNode.state)
	#print(actions)
	if not actions:
		return ("failure", float('inf'))
	children = []
	for action in actions:
		aStarCounter += 1
		(childState, stepCost) = takeActionF(parentNode.state, action)
		h = hF(childState)
		g = parentNode.g + stepCost
		f = max(h+g, parentNode.f)
		childNode = Node(state=childState, f=f, g=g, h=h)
		children.append(childNode)
	while True:
		# find best child
		children.sort(key = lambda n: n.f) # sort by f value
		bestChild = children[0]
		if bestChild.f > fmax:
			return ("failure",bestChild.f)
		# next lowest f value
		alternativef = children[1].f if len(children) > 1 else float('inf')
		# expand best child, reassign its f value to be returned value
		result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
		if result is not "failure":
			aStarDepth += 1
			result.insert(0,parentNode.state)
			return (result, bestChild.f)
          
def takeActionF_simple(state, action):
	return action

def goalTestF_simple(state, goal):
	return state == goal

def h_simple(state, goal):
	return 0

def h1_8p(state, goal):
	return 1
 
def h3_8p(state, goal):
	sIndex = state.index(0)
	gIndex = goal.index(0)
	return math.fabs(sIndex - gIndex) / 2
	
def h2_8p(state, goal):
	sIndex = state.index(0)
	gIndex = goal.index(0)
	
	c = columnDiff(sIndex, gIndex)
	r = rowDiff(sIndex, gIndex)
	
	return c + r
	
def columnDiff(s, g):
	sColumn = 0
	gColumn = 0
	
	if s == 0 or s == 3 or s == 6:
		sColumn = 0
	elif s == 1 or s == 4 or s == 7:
		sColumn = 1
	else:
		sColumn = 2
		
	if g == 0 or g == 3 or g == 6:
		sColumn = 0
	elif g == 1 or g == 4 or g == 7:
		gColumn = 1
	else:
		gColumn = 2
		
	return math.fabs(gColumn - sColumn)
	

def rowDiff(s, g):
	sRow = 0
	gRow = 0
	
	if s == 0 or s == 1 or s == 2:
		sRow = 0
	elif s == 3 or s == 4 or s == 5:
		sRow = 1
	else:
		sRow = 2
		
	if g == 0 or g == 1 or g ==2:
		gRow = 0
	elif g == 3 or g == 4 or g == 5:
		gRow = 1
	else:
		gRow = 2
		
	return math.fabs(gRow - sRow)


def runExperiment(goalState1, goalState2, goalState3, heuristics = []):
	global aStarDepth
	global aStarCounter
	startState = [1, 2, 3, 4, 0, 5, 6, 7, 8]
	print("\t{}\t\t{}\t\t{}\t".format(goalState1, goalState2, goalState3))
	print("Algorithm      Depth   Nodes   EBF\t\t  Depth   Nodes   EBF\t\t\t Depth   Nodes   EBF")
  
	# Iterative Deepening
	l = iterativeDeepeningSearch(startState, goalState1, actionsF_8p, takeActionF_8p, 999999)
	print("\tIDS\t{}\t{}\t{}".format(itsDepth, itsTotal, round(ebf(itsTotal, itsDepth, 0.01), 3)), end='')
  
	l = iterativeDeepeningSearch(startState, goalState2, actionsF_8p, takeActionF_8p, 999999)
	print("\t\t{}\t{}\t{}".format(itsDepth, itsTotal, round(ebf(itsTotal, itsDepth, 0.01), 3)), end='')
  
	l = iterativeDeepeningSearch(startState, goalState3, actionsF_8p, takeActionF_8p, 999999)
	print("\t\t\t{}\t{}\t{}".format(itsDepth, itsTotal, round(ebf(itsTotal, itsDepth, 0.01), 3)))

	counter = 1
	for H in heuristics:
		aStarSearch(startState, actionsF_8p, takeActionF_8p, lambda s: goalTestF_simple(s, goalState1), lambda s: H(s, goalState1))
		print("\tA*H{}\t{}\t{}\t{}".format(counter, aStarDepth, aStarCounter, round(ebf(aStarCounter, aStarDepth, 0.01), 3)), end='')
		
		aStarSearch(startState, actionsF_8p, takeActionF_8p, lambda s: goalTestF_simple(s, goalState2), lambda s: H(s, goalState2))
		print("\t\t{}\t{}\t{}".format(aStarDepth, aStarCounter, round(ebf(aStarCounter, aStarDepth, 0.01), 3)), end='')
		
		aStarSearch(startState, actionsF_8p, takeActionF_8p, lambda s: goalTestF_simple(s, goalState3), lambda s: H(s, goalState3))
		print("\t\t\t{}\t{}\t{}".format(aStarDepth, aStarCounter, round(ebf(aStarCounter, aStarDepth, 0.01), 3)))
		counter += 1
		
if __name__ == "__main__":
	runExperiment([1, 2, 3, 4, 0, 5, 6, 7, 8], [1, 2, 3, 4, 5, 8, 6, 0, 7], [1, 0, 3, 4, 5, 8, 2, 6, 7], [h1_8p, h2_8p, h3_8p])


	[1, 2, 3, 4, 0, 5, 6, 7, 8]		[1, 2, 3, 4, 5, 8, 6, 0, 7]		[1, 0, 3, 4, 5, 8, 2, 6, 7]	
Algorithm      Depth   Nodes   EBF		  Depth   Nodes   EBF			 Depth   Nodes   EBF
	IDS	0	0	0		3	43	3.101			11	225850	2.949
	A*H1	0	0	0		3	116	4.477			11	643246	3.18
	A*H2	0	0	0		3	51	3.313			11	100046	2.731
	A*H3	0	0	0		3	32	2.721			11	214662	2.941


The above output is the table of stats produced by all three algorithms containing Depth, Number of Nodes expanded, and EBF for each of the three known end states

#### Here is an example of EBF running

In [35]:
ebf(10, 3)

1.6494140625

This provides that, on average, each node will generate ~1.65 successors

The smallest argument values should be a depth of 0, and 1 node.

In [36]:
ebf(1, 0)

1.0

### The following are example test cases for testing the EBF function
* Results are assumed within a +/- 10% margin 

In [37]:
ebf(2, 1)

0.9921875

In [38]:
ebf(2, 1, precision=0.000001)

1.0000009536743164

In [39]:
ebf(200000, 5)

11.464426517486572

In [40]:
ebf(200000, 50)

1.2350082397460938

Here is a simple example using our usual simple graph search.

In [41]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                            